#### Hypothesis


<span style='font-family:TH Baijam;font-size: 135%'>สมมติฐาน(Hypothesis) คือ สิ่งที่ได้จากกระบวนการเรียนรู้เพื่อคาดเดาแนวคิด ซึ่งอาจจะอยู่ในรูปของสมการหรือเป็นรูปแบบอื่นๆ ที่เราสามารถนำไปใช้ในการทำนายได้
ตัวอย่างเช่น เรามีข้อมูลของ4วันที่ประกอบไปด้วย Sky,Temp,Humid,Wind,Water,Forecast,PlayTennis ซึ่งเป็นค่าที่เราต้องการจะทำนายว่าเมื่อมีข้อมูลเหล่านี้(ข้อมูลอันดับที่1-6)เข้ามา จะมีสามาถเล่นเทนนิสได้หรือไม่ ซึ่งเราจะสมมติฐานว่า ข้อมูลที่เรามีนั้นจะมีความสัมพันธ์กัน และสามารถนำมาทำนายได้</span>


# Find-S algorithm

<span style='font-family:TH Baijam;font-size: 135%'>
S มาจากคำว่า Specific โดย algorithm นี้จะเริ่มต้นจาก The Most Specific Hypothesis ซึ่งก็คือ สมมติฐานที่ไม่รองรับตัวอย่างใดๆ (Null Hypothesis) หลังจากนั้นก็ทำการขยายสมมติฐานให้ครอบคลุมตัวอย่างฝึกฝนทั้งหมด โดย algorithm เป็นดังนี้
</span>

![Find-S algorithm](https://i.imgur.com/BLuvbPd.jpeg)


In [1]:
# Find-S Algorithm
examples = [['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Yes'],
      ['Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Yes'],
      ['Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'No'],
      ['Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Yes']]


def findS(examples):
    hypothesis = ['0'] * (len(examples[0])-1)
    for example in examples:
        if example[-1] == 'Yes':
            for i in range(len(example)-1):
                if hypothesis[i] == '0':
                    hypothesis[i] = example[i]
                elif hypothesis[i] != example[i]:
                    hypothesis[i] = '?'
    return hypothesis


print(findS(examples))


['Sunny', 'Warm', '?', 'Strong', '?', '?']


The goal of the Find-S algorithm is to find a hypothesis that will correctly classify all the positive examples in the training data.

In the context of the Find-S algorithm, `['Sunny','Warm','?','Strong','?','?']` represents a hypothesis. Each element of this list corresponds to a feature or attribute of the data, and the '?' character stands for a "don't care" condition, meaning that any value is acceptable for that attribute


---


# List-Then-Eliminate algorithm

<span style='font-family:TH Baijam;font-size: 135%'>
List-Then-Eliminate algorithm จะได้ผลลัพธ์เป็นเซตของสมมติฐานที่สอดคล้องกับตัวอย่างฝึกฝนทั้งหมด ซึ่งเราเรียกว่า Version Space
การหาสมมติฐานที่เป็นไปได้ทั้งหมดเป็นงานที่หนักในบางปัญหาที่มีข้อมูลจำนวนมาก ทำให้ algorithm นี้ไม่เหมาะสมในการใช้งานจริง<br>
- Version Space เป็นเซตของสมมติฐานที่อยู่ระหว่างสมมติฐานที่เฉพาะเจาะจงเกินไป และสมมติฐานที่ทั่วไปเกินไป<br>
- สมมติฐานที่เฉพาะเจาะจงเกินไป เช่น "คนที่อายุ 18 ปีพอดีชอบเพลงป๊อปและคนที่อายุ 19 ปีพอดีชอบเพลงร็อค" มีแนวโน้มที่เฉพาะเจาะจงเกินไป<br>
- สมมติฐานที่ทั่วไปเกินไป เช่น "คนที่อายุ 18 ปีทุกคนชอบเพลงป๊อป" มีแนวโน้มที่ทั่วไปเกินไป
</span>

algorithm:

1. Initialize the list of hypotheses to contain all hypotheses in the hypothesis space.
2. For each training example, eliminate from the list any hypothesis inconsistent with the training example.
3. Output the list of hypotheses.

example ; assume we have the following dataset

```py
ex = [['Sunny','Warm','Normal','Strong','Warm','Same','Yes'],
        ['Rainy','Cold','High','Strong','Warm','Change','No']]
```

At the start, we have not seen any examples, so all hypotheses are possible. As we start seeing examples, we'll start eliminating hypotheses that do not match the examples.

After seeing the first example `['Sunny','Warm','Normal','Strong','Warm','Same','Yes']`, we eliminate all hypotheses that **do not predict** 'Yes' for this example.

The third example `['Rainy','Cold','High','Strong','Warm','Change','No']` is a negative example. We eliminate all hypotheses that **predict** 'Yes' for this example.

At the end, we are left with a set of hypotheses that are consistent with all the examples we have seen.


In [ ]:
# Code by Parinya Sanguansat
import numpy as np

def ListElim(X, T):
    X = np.array(X)
    T = np.array(T)
    n = X.shape[1]
    A = []
    for i in range(n):
        A.append(sorted(list(set(X[:, i]))))

    H = []
    t = []
    i = 1
    idx_data = [0]*n
    while True:
        h = []
        for j in range(n):
            h.append(A[j][idx_data[j]])

        for tt in np.unique(T):
            if isConsist(X, T, h, tt):
                H.append(h)
                t.append(tt)
                print(i, h, tt)
                i += 1

        idx_data[-1] += 1
        letter_index = n-1
        while idx_data[letter_index] > len(A[letter_index])-1:
            idx_data[letter_index] = 0
            letter_index -= 1
            if letter_index < 0:
                return H, t
            idx_data[letter_index] += 1

def isConsist(X, T, h, t):
    for i in range(len(X)):
        if h == list(X[i]) and t != T[i]:
            return False
    return True


ex = [['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same'],
      ['Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same'],
      ['Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change'],
      ['Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change']]
T = ['Yes', 'Yes', 'No', 'Yes']
H, t = ListElim(ex, T)

This is my implementation of the List-Then-Eliminate algorithm

In [ ]:
# My version, Phaiboon Jaradnaparatana
from itertools import product
import numpy as np

examples = np.array([['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Yes'],
                    ['Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Yes'],
                    ['Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'No'],
                    ['Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Yes']])

# get unique values for each attribute
unique_values = [np.unique(examples[:, i]) for i in range(examples.shape[1])]
# generate all possible hypotheses
hypotheses = list(product(*unique_values))

# eliminate hypotheses that are inconsistent with the examples
remaining_hypotheses = []
for h in hypotheses:
    eliminate = False  # flag to mark whether to eliminate the current hypothesis
    for e in examples:
        if all(h[i] == e[i] for i in range(len(h)-1)) and h[-1] != e[-1]:
            eliminate = True
            break
    if not eliminate:
        remaining_hypotheses.append(h)
hypotheses = remaining_hypotheses

print(f"All possible hypotheses is:{len(hypotheses)}")
for h in hypotheses:
    print(h)

---

# Candidate Elimination algorithm

<span style='font-family:TH Baijam;font-size: 135%'>
